In [11]:
import sys
import os
import subprocess

import four_gamete, vcf_split_pysam, vcf_filter, vcf_calc, vcf_phase, stat_sampler, vcf_split
from informative_loci_filter import filter_bed_regions
#import filter_stat
import pysam 

print ("Imports complete")

Imports complete


In [51]:
#Set up directories and filepaths, run on all restarts
work_dir='toy_files/'
#data_dir='/media/ccgg/ppp_sample_data/'
data_dir=work_dir
vcf_dir = work_dir+'vcfs/'


main_vcf_name = data_dir+'pan_chr20.vcf.gz'
filtered_vcf_name = vcf_dir+'pan_chr20_filtered_missing.vcf.gz'
stat_file_name = vcf_dir+'fst_regions.bed'
model_file = data_dir+'great_ape.model'
int_bed_file = vcf_dir+'regions_for_sampling.bed'
target_loci_file = vcf_dir+'target_loci.bed'
ima_input_file = vcf_dir+'test_run_input.ima.u'
#subsamp_bed_file = work_dir+'great_ape_genome2/5k_sample.bed'
#logfile = '/home/jared/testpppj.log'

loci=200

region_files = [vcf_dir+'Sampled_nonmissing/out_'+str(i)+'.vcf.gz' for i in range(loci)]
phased_files = [vcf_dir+'Phased/phased_'+str(i)+'.vcf' for i in range(loci)]
fourg_files = [vcf_dir+'four_gamete/Sample_'+str(i)+'.vcf' for i in range(loci)]
passed_files = []

In [14]:
#Set up directory structure, only needs to be run once
if not os.path.exists(vcf_dir):
    os.makedirs(vcf_dir)
    os.makedirs(vcf_dir+'four_gamete/')
    os.makedirs(vcf_dir+'Sampled_nonmissing/')
    os.makedirs(vcf_dir+'Phased/')

In [25]:
#Creates VCF filtered for no missing data and biallelic sites
vcf_filter.run(['--vcf', main_vcf_name, '--filter-max-missing', '1.0', '--model-file',model_file,
                '--model','2Pop', '--filter-min-alleles', '2', '--filter-max-alleles', '2', '--out-format',
                'vcf.gz', '--out', filtered_vcf_name, '--overwrite'])

pysam.tabix_index(filtered_vcf_name,preset='vcf')
print("Filtering complete")

## RUN AT TERMINAL 

TypeError: run() takes 0 positional arguments but 1 was given

In [52]:
#Calculates f_st statistics across genome
vcf_calc.run(['--vcf', filtered_vcf_name, '--out', stat_file_name,
              '--calc-statistic', 'windowed-weir-fst', '--model', '2Pop', '--statistic-window-size',
              '10000', '--statistic-window-step', '10000', '--model-file', model_file, '--overwrite'])
print("Stat calculation complete")



Stat calculation complete


In [27]:
#Selects subset of regions for fast sampling
filter_bed_regions(['--vcf',filtered_vcf_name,'--bed',stat_file_name,
                    '--remove-indels','--minsites','3','--keep-full-line','--out',int_bed_file,
                    '--randcount','5000','--remove-multi'])
print("BED regions selected")
## RUN AT TERMINAL 

TypeError: filter_bed_regions() takes 0 positional arguments but 1 was given

In [28]:
stat_sampler.run(['--statistic-file',int_bed_file,'--out',target_loci_file,'--sampling-scheme','uniform',
                 '--uniform-bins','5','--sample-size',str(loci),'--calc-statistic','windowed-weir-fst','--overwrite'])

## RUN AT TERMINAL 

TypeError: run() takes 0 positional arguments but 1 was given

In [53]:
#Uniformly sample regions for subset of 200 loci
#vcf_sampler.run(['--vcf', filtered_vcf_name, '--statistic-file',
#                 target_loci_file, '--out-format', 'vcf', '--calc-statistic', 'windowed-weir-fst',
#                 '--sampling-scheme', 'uniform', '--uniform-bins', '5', '--out-dir',
#                 work_dir + 'great_ape_genome2/Sample_Files', '--overwrite'])
#vcf_split.run(['--vcf',filtered_vcf_name,'--split-method','statistic-file','--out-format','vcf','--out-prefix',
#              vcf_dir+'Sampled_nonmissing/Sample','--split-file',target_loci_file])
#vcf_split_pysam.vcf_region_write([filtered_vcf_name,'--rl',target_loci_file,
#                                  '--remove-indels','--remove-multi','--informative-count','2'])

print("Sampling complete")

## RUN SEPARATELY IN TERMINAL 

Exception: Error: Could not open VCF Output file: Split_VCFs/toy_files/vcfs/Sampled_nonmissing/Sample_0.recode.vcf

In [54]:
import subprocess

In [55]:
#Phase locus
for i in range(loci):
    subprocess.run(['python','vcf_phase.py','--vcf',region_files[i],'--phase-algorithm','shapeit','--out',
                   phased_files[i],'--out-format','vcf','--overwrite'])
print ("Phasing done")



Phasing done


In [56]:
#Subsample locus for four-gamete compatible interval, if no subregion returned, do not use VCF
passed_files = []
for i in range(loci):
    ret = four_gamete.sample_fourgametetest_intervals(['--vcfs', phased_files[i], '--out',
                                                       fourg_files[i], '--4gcompat', '--reti', '--right',
                                                       '--numinf', '2'])
    if ret[0] is not None:
        passed_files.append(fourg_files[i])
print ("Four gamete regions selected for %d loci"%(len(passed_files)))

Four gamete regions selected for 127 loci


In [66]:
import vcf_to_ima

In [65]:
with open('toy_files/vcfs/filename.txt', 'w') as o:
    for path in passed_files:
        o.write(path + '\n')

In [67]:
#Create IMa input file
ima_args = ['--vcfs']
ima_args.extend(passed_files)
ima_args.extend(['--model-file', model_file, '--model','2Pop','--out', work_dir + 'ima_all_loci.ima.u'])

vcf_to_ima.vcf_to_ima(ima_args)
print ("IMa input created")

usage: ipykernel_launcher.py [-h] [--vcf VCFNAME]
                             [--vcfs VCFLIST [VCFLIST ...]] [--ref REFNAME]
                             [--bed GENENAME] [--pop POPNAME] [--zero-ho]
                             [--zero-closed] [--keep-indels]
                             [--remove-multiallele]
                             [--remove-missing REMOVE_MISSING] [--parsecpg]
                             [--noseq] [--trim-to-ref-length]
                             [--output OUTPUT_NAME] [--gene-col GENE_COL]
                             [--compress-vcf] [--conf CONFIG_NAME]
                             [--no-ref-check] [--poptag POPTAG]
                             [--mutrate MUTRATE]
                             [--inheritance-scalar INHET_SC] [--fasta]
                             [--multi-out MULTI_OUT]
ipykernel_launcher.py: error: unrecognized arguments: --model-file toy_files/great_ape.model --model 2Pop


SystemExit: 2

/home/tu.temple.edu/tuk32868/miniconda3/envs/ppp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [68]:
#Admixture analysis, optional
import convert, admixture, graph_plotter
phased_string = ' '.join(phased_files)
loci_vcf = vcf_dir+'Phased/phased_merged.vcf.gz'
concatcall = subprocess.Popen('vcf-concat '+phased_string+ ' | bgzip -c > '+loci_vcf, shell=True,stdout=subprocess.PIPE)
temp_out, temp_err = concatcall.communicate()
convert.run(['--vcf',loci_vcf,'--out-format','binary-ped','--out-prefix',vcf_dir+'great_ape','--overwrite'])
admixture.run(['--file',vcf_dir+'great_ape.bed','--pop','2'])
graph_plotter.bar_plot(vcf_dir+'great_ape.2.Q')
print ("Plots created")



ModuleNotFoundError: No module named 'convert'